In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query

from datetime import datetime

종목코드열람

In [2]:
shcode_list = query.query_shcode_list_today()
today = datetime.now().strftime("%Y%m%d")

In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [5]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 500
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 5:
            average_volume_test[ind] = 1
            
    return average_volume_test

In [10]:
v = '335870'

daily_data = query.query_ohlcv(v, fr = '20181201',to = today)

In [13]:
daily_data.shape[0]

0

In [9]:
week_data = query.query_ohlcv_t1102(shcode = v, to = today)

if week_data['date'].iloc[-1] >= daily_data['date'].iloc[-1]:
    daily_data = daily_data.append(week_data).reset_index(drop = True)
else:
    pass

#array_daily = np.array(daily_data, dtype = np.int64)

IndexError: single positional indexer is out-of-bounds

In [ ]:
daily_data.tail()

In [18]:
week_data.tail()

,date,open,high,low,close,volume
0,20191209,7810,7860,7500,7580,280557


#최종

In [14]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.05
volume_money = 20000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    #daily_data = query.query_ohlcv(v, to = today)
    daily_data = query.query_ohlcv(v, fr = '20181201',to = today)
    if daily_data.shape[0] == 0:
        continue
    
    week_data = query.query_ohlcv_t1102(shcode = v, to = today)
    
    if week_data['date'].iloc[-1] >= daily_data['date'].iloc[-1]:
        daily_data = daily_data.append(week_data).reset_index(drop = True)
    else:
        pass

    array_daily = np.array(daily_data, dtype = np.int64)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v4' +'.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-12-10_12:46:25 : , 0 , 000020 , 동화약품 -------------------  
O_1. ratio_test 통과 0 , 000020 , 동화약품
X_2. total_volume_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-12-10_12:46:25 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-12-10_12:46:26 : , 2 , 000050 , 경방 -------------------  
O_1. ratio_test 통과 2 , 000050 , 경방
X_2. total_volume_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-12-10_12:46:26 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-12-10_12:46:27 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-12-10_12:46:27 : , 5 , 000075 , 삼양홀딩스우 -------------------  
O_1. ratio_test 통과 5 , 000075 , 삼양홀딩스우
X_2. total_volume_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-12-10_12:46:28 : , 6 , 000080 , 하이트진로 -----------------

O_1. ratio_test 통과 40 , 000640 , 동아쏘시오홀딩스
X_2. total_volume_test 없음 40 , 000640 , 동아쏘시오홀딩스
------------------- test 시작_2019-12-10_12:46:48 : , 41 , 000650 , 천일고속 -------------------  
X_1. ratio_test 없음 41 , 000650 , 천일고속
------------------- test 시작_2019-12-10_12:46:48 : , 42 , 000660 , SK하이닉스 -------------------  
O_1. ratio_test 통과 42 , 000660 , SK하이닉스
O_2. total_volume_test 통과 42 , 000660 , SK하이닉스
X_3. average_volume_test 없음 42 , 000660 , SK하이닉스
------------------- test 시작_2019-12-10_12:46:49 : , 43 , 000670 , 영풍 -------------------  
O_1. ratio_test 통과 43 , 000670 , 영풍
O_2. total_volume_test 통과 43 , 000670 , 영풍
X_3. average_volume_test 없음 43 , 000670 , 영풍
------------------- test 시작_2019-12-10_12:46:49 : , 44 , 000680 , LS네트웍스 -------------------  
O_1. ratio_test 통과 44 , 000680 , LS네트웍스
X_2. total_volume_test 없음 44 , 000680 , LS네트웍스
------------------- test 시작_2019-12-10_12:46:50 : , 45 , 000700 , 유수홀딩스 -------------------  
O_1. ratio_test 통과 45 , 000700 , 유수홀딩스
X_2. total_volume

O_1. ratio_test 통과 82 , 001340 , 백광산업
O_2. total_volume_test 통과 82 , 001340 , 백광산업
O_3. average_volume_test 통과 82 , 001340 , 백광산업
------------------- test 완료 : , 82 , 001340 , 백광산업 ------------------- 
------------------- test 시작_2019-12-10_12:47:11 : , 83 , 001360 , 삼성제약 -------------------  
O_1. ratio_test 통과 83 , 001360 , 삼성제약
O_2. total_volume_test 통과 83 , 001360 , 삼성제약
O_3. average_volume_test 통과 83 , 001360 , 삼성제약
------------------- test 완료 : , 83 , 001360 , 삼성제약 ------------------- 
------------------- test 시작_2019-12-10_12:47:12 : , 84 , 001380 , SG충방 -------------------  
O_1. ratio_test 통과 84 , 001380 , SG충방
O_2. total_volume_test 통과 84 , 001380 , SG충방
O_3. average_volume_test 통과 84 , 001380 , SG충방
------------------- test 완료 : , 84 , 001380 , SG충방 ------------------- 
------------------- test 시작_2019-12-10_12:47:13 : , 85 , 001390 , KG케미칼 -------------------  
O_1. ratio_test 통과 85 , 001390 , KG케미칼
X_2. total_volume_test 없음 85 , 001390 , KG케미칼
------------------- test 시작_2

O_1. ratio_test 통과 122 , 001940 , KISCO홀딩스
X_2. total_volume_test 없음 122 , 001940 , KISCO홀딩스
------------------- test 시작_2019-12-10_12:47:33 : , 123 , 002020 , 코오롱 -------------------  
O_1. ratio_test 통과 123 , 002020 , 코오롱
O_2. total_volume_test 통과 123 , 002020 , 코오롱
O_3. average_volume_test 통과 123 , 002020 , 코오롱
------------------- test 완료 : , 123 , 002020 , 코오롱 ------------------- 
------------------- test 시작_2019-12-10_12:47:34 : , 124 , 002025 , 코오롱우 -------------------  
O_1. ratio_test 통과 124 , 002025 , 코오롱우
X_2. total_volume_test 없음 124 , 002025 , 코오롱우
------------------- test 시작_2019-12-10_12:47:34 : , 125 , 002030 , 아세아 -------------------  
X_1. ratio_test 없음 125 , 002030 , 아세아
------------------- test 시작_2019-12-10_12:47:35 : , 126 , 002070 , 남영비비안 -------------------  
O_1. ratio_test 통과 126 , 002070 , 남영비비안
O_2. total_volume_test 통과 126 , 002070 , 남영비비안
O_3. average_volume_test 통과 126 , 002070 , 남영비비안
------------------- test 완료 : , 126 , 002070 , 남영비비안 ------------------

O_1. ratio_test 통과 159 , 002790 , 아모레G
O_2. total_volume_test 통과 159 , 002790 , 아모레G
X_3. average_volume_test 없음 159 , 002790 , 아모레G
------------------- test 시작_2019-12-10_12:47:55 : , 160 , 002795 , 아모레G우 -------------------  
O_1. ratio_test 통과 160 , 002795 , 아모레G우
X_2. total_volume_test 없음 160 , 002795 , 아모레G우
------------------- test 시작_2019-12-10_12:47:56 : , 161 , 002810 , 삼영무역 -------------------  
O_1. ratio_test 통과 161 , 002810 , 삼영무역
X_2. total_volume_test 없음 161 , 002810 , 삼영무역
------------------- test 시작_2019-12-10_12:47:57 : , 162 , 002820 , 선창산업 -------------------  
X_1. ratio_test 없음 162 , 002820 , 선창산업
------------------- test 시작_2019-12-10_12:47:57 : , 163 , 002840 , 미원상사 -------------------  
O_1. ratio_test 통과 163 , 002840 , 미원상사
X_2. total_volume_test 없음 163 , 002840 , 미원상사
------------------- test 시작_2019-12-10_12:47:58 : , 164 , 002870 , 신풍제지 -------------------  
O_1. ratio_test 통과 164 , 002870 , 신풍제지
X_2. total_volume_test 없음 164 , 002870 , 신풍제지
---------------

...

O_1. ratio_test 통과 2776 , 247540 , 에코프로비엠
O_2. total_volume_test 통과 2776 , 247540 , 에코프로비엠
X_3. average_volume_test 없음 2776 , 247540 , 에코프로비엠
------------------- test 시작_2019-12-10_13:12:5 : , 2777 , 250000 , 보라티알 -------------------  
O_1. ratio_test 통과 2777 , 250000 , 보라티알
O_2. total_volume_test 통과 2777 , 250000 , 보라티알
O_3. average_volume_test 통과 2777 , 250000 , 보라티알
------------------- test 완료 : , 2777 , 250000 , 보라티알 ------------------- 
------------------- test 시작_2019-12-10_13:12:5 : , 2778 , 250060 , 모비스 -------------------  
O_1. ratio_test 통과 2778 , 250060 , 모비스
O_2. total_volume_test 통과 2778 , 250060 , 모비스
O_3. average_volume_test 통과 2778 , 250060 , 모비스
------------------- test 완료 : , 2778 , 250060 , 모비스 ------------------- 
------------------- test 시작_2019-12-10_13:12:6 : , 2779 , 250930 , 예선테크 -------------------  
O_1. ratio_test 통과 2779 , 250930 , 예선테크
O_2. total_volume_test 통과 2779 , 250930 , 예선테크
O_3. average_volume_test 통과 2779 , 250930 , 예선테크
------------------- test 

O_1. ratio_test 통과 2809 , 263750 , 펄어비스
O_2. total_volume_test 통과 2809 , 263750 , 펄어비스
X_3. average_volume_test 없음 2809 , 263750 , 펄어비스
------------------- test 시작_2019-12-10_13:12:22 : , 2810 , 263770 , 유에스티 -------------------  
O_1. ratio_test 통과 2810 , 263770 , 유에스티
X_2. total_volume_test 없음 2810 , 263770 , 유에스티
------------------- test 시작_2019-12-10_13:12:23 : , 2811 , 263800 , 데이타솔루션 -------------------  
O_1. ratio_test 통과 2811 , 263800 , 데이타솔루션
O_2. total_volume_test 통과 2811 , 263800 , 데이타솔루션
O_3. average_volume_test 통과 2811 , 263800 , 데이타솔루션
------------------- test 완료 : , 2811 , 263800 , 데이타솔루션 ------------------- 
------------------- test 시작_2019-12-10_13:12:23 : , 2812 , 263810 , 상신전자 -------------------  
O_1. ratio_test 통과 2812 , 263810 , 상신전자
O_2. total_volume_test 통과 2812 , 263810 , 상신전자
O_3. average_volume_test 통과 2812 , 263810 , 상신전자
------------------- test 완료 : , 2812 , 263810 , 상신전자 ------------------- 
------------------- test 시작_2019-12-10_13:12:24 : , 2813 , 263

O_1. ratio_test 통과 2846 , 285490 , 노바텍
O_2. total_volume_test 통과 2846 , 285490 , 노바텍
O_3. average_volume_test 통과 2846 , 285490 , 노바텍
------------------- test 완료 : , 2846 , 285490 , 노바텍 ------------------- 
------------------- test 시작_2019-12-10_13:12:42 : , 2847 , 286750 , 나노브릭 -------------------  
O_1. ratio_test 통과 2847 , 286750 , 나노브릭
O_2. total_volume_test 통과 2847 , 286750 , 나노브릭
X_3. average_volume_test 없음 2847 , 286750 , 나노브릭
------------------- test 시작_2019-12-10_13:12:42 : , 2848 , 287410 , 유안타제3호스팩 -------------------  
X_1. ratio_test 없음 2848 , 287410 , 유안타제3호스팩
------------------- test 시작_2019-12-10_13:12:43 : , 2849 , 288620 , 에스퓨얼셀 -------------------  
O_1. ratio_test 통과 2849 , 288620 , 에스퓨얼셀
O_2. total_volume_test 통과 2849 , 288620 , 에스퓨얼셀
O_3. average_volume_test 통과 2849 , 288620 , 에스퓨얼셀
------------------- test 완료 : , 2849 , 288620 , 에스퓨얼셀 ------------------- 
------------------- test 시작_2019-12-10_13:12:43 : , 2850 , 289010 , 아이스크림에듀 -------------------  
O_1. ratio_t

O_1. ratio_test 통과 2879 , 306040 , 에스제이그룹
O_2. total_volume_test 통과 2879 , 306040 , 에스제이그룹
X_3. average_volume_test 없음 2879 , 306040 , 에스제이그룹
------------------- test 시작_2019-12-10_13:12:59 : , 2880 , 306620 , DB금융스팩6호 -------------------  
X_1. ratio_test 없음 2880 , 306620 , DB금융스팩6호
------------------- test 시작_2019-12-10_13:13:0 : , 2881 , 307070 , SK4호스팩 -------------------  
X_1. ratio_test 없음 2881 , 307070 , SK4호스팩
------------------- test 시작_2019-12-10_13:13:0 : , 2882 , 307160 , 하나머스트제6호스팩 -------------------  
O_1. ratio_test 통과 2882 , 307160 , 하나머스트제6호스팩
O_2. total_volume_test 통과 2882 , 307160 , 하나머스트제6호스팩
O_3. average_volume_test 통과 2882 , 307160 , 하나머스트제6호스팩
------------------- test 완료 : , 2882 , 307160 , 하나머스트제6호스팩 ------------------- 
------------------- test 시작_2019-12-10_13:13:1 : , 2883 , 307180 , 신영스팩4호 -------------------  
O_1. ratio_test 통과 2883 , 307180 , 신영스팩4호
O_2. total_volume_test 통과 2883 , 307180 , 신영스팩4호
O_3. average_volume_test 통과 2883 , 307180 , 신영스팩4호
-----

O_1. ratio_test 통과 2917 , 323210 , 이베스트이안스팩1호
O_2. total_volume_test 통과 2917 , 323210 , 이베스트이안스팩1호
X_3. average_volume_test 없음 2917 , 323210 , 이베스트이안스팩1호
------------------- test 시작_2019-12-10_13:13:20 : , 2918 , 323230 , 신한제5호스팩 -------------------  
O_1. ratio_test 통과 2918 , 323230 , 신한제5호스팩
O_2. total_volume_test 통과 2918 , 323230 , 신한제5호스팩
X_3. average_volume_test 없음 2918 , 323230 , 신한제5호스팩
------------------- test 시작_2019-12-10_13:13:20 : , 2919 , 323280 , 신영스팩5호 -------------------  
O_1. ratio_test 통과 2919 , 323280 , 신영스팩5호
O_2. total_volume_test 통과 2919 , 323280 , 신영스팩5호
X_3. average_volume_test 없음 2919 , 323280 , 신영스팩5호
------------------- test 시작_2019-12-10_13:13:21 : , 2920 , 323940 , 케이비제18호스팩 -------------------  
X_1. ratio_test 없음 2920 , 323940 , 케이비제18호스팩
------------------- test 시작_2019-12-10_13:13:21 : , 2921 , 328380 , 미래에셋대우스팩3호 -------------------  
X_1. ratio_test 없음 2921 , 328380 , 미래에셋대우스팩3호
------------------- test 시작_2019-12-10_13:13:22 : , 2922 , 329560 , 상상인이